In [57]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from tensorflow import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.losses import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [58]:
# Session & seed configurations
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

tf.keras.utils.set_random_seed(1)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

In [59]:
stock = 'SPY'
data = pd.read_feather(f'C:/Users/PieterVerloop/Downloads/Papers DS/0000 DATA/stock-price-forecasting-dcgan-main/stock-price-forecasting-dcgan-main/data/XGBoost10/df_{stock}_XGBoost10.feather')
data = data.set_index(['date'])
data

,close,open,high,low,volume,volume_vwap,volume_adi,trend_ichimoku_conv,volatility_dch,momentum_kama,trend_ema_fast,trend_visual_ichimoku_b,volatility_kcl,volume_obv,volume_nvi
date,,,,,,,,,,,,,,,
20000414,136.000000,142.625000,142.812500,133.500000,29604000,147.031319,1.579690e+07,143.304688,155.750000,145.696069,146.695432,140.484375,143.536979,-6.734020e+07,1013.245943
20000417,140.750000,135.187500,140.750000,134.687500,23918200,145.691280,3.971510e+07,143.304688,155.750000,145.181389,145.780750,140.484375,142.020312,-4.342200e+07,1048.635048
20000418,144.468750,140.562500,144.468750,139.781250,11069200,145.303264,5.078430e+07,143.304688,155.750000,145.155602,145.578904,140.484375,142.185937,-3.235280e+07,1076.340991
20000419,143.125000,144.500000,145.125000,142.531250,6553700,145.018813,4.723109e+07,143.304688,155.750000,145.076586,145.201380,140.484375,141.683854,-3.890650e+07,1066.329600
20000420,143.812500,143.562500,143.937500,142.375000,8537600,144.645290,5.440267e+07,143.304688,155.750000,145.018741,144.987706,140.484375,141.094792,-3.036890e+07,1066.329600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,477.260010,472.059998,477.309998,472.010010,56808600,465.335830,3.847010e+10,464.225006,477.309998,463.764997,466.795627,448.504990,459.408670,1.254043e+10,356405.200996
20211228,476.869995,477.720001,478.809998,476.059998,47274600,465.826243,3.845067e+10,464.975006,478.809998,464.246921,468.345530,448.504990,460.507337,1.249315e+10,356113.948332
20211229,477.480011,476.980011,478.559998,475.920013,54503000,466.187341,3.846058e+10,464.975006,478.809998,465.065999,469.750835,448.649994,462.212673,1.254766e+10,356113.948332


In [60]:
close_prices = data['close']
values = close_prices.values
training_data_len = math.ceil(len(values)* 0.8)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(5, len(train_data)):
    x_train.append(train_data[i-5:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [61]:
test_data = scaled_data[training_data_len-5: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(5, len(test_data)):
  x_test.append(test_data[i-5:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [62]:
# Basic model
model = keras.Sequential()
model.add(layers.LSTM(32, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.LSTM(64, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(1))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 5, 32)             4352      
                                                                 
 lstm_15 (LSTM)              (None, 64)                24832     
                                                                 
 dense_14 (Dense)            (None, 25)                1625      
                                                                 
 dense_15 (Dense)            (None, 1)                 26        
                                                                 
Total params: 30,835
Trainable params: 30,835
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Same as in GAN, but is worse than basic
# model = tf.keras.Sequential()
# model.add(layers.Dense(32, input_shape=(x_train.shape[1], 1)))
# model.add(layers.LeakyReLU(alpha=0.1))
# model.add(layers.LSTM(64, activation='relu', return_sequences=False, dropout=0.2, recurrent_dropout=0.0))                                 
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='linear'))
# model.add(layers.LeakyReLU(alpha=0.1))
# model.add(layers.Dense(32, activation='linear'))
# model.add(layers.LeakyReLU(alpha=0.1))
# model.add(layers.Dense(1))
# model.summary()

In [64]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 5, epochs=3)

Epoch 1/3
874/874 [==============================] - 6s 3ms/step - loss: 4.5242e-04
Epoch 2/3
874/874 [==============================] - 3s 3ms/step - loss: 5.5628e-05
Epoch 3/3
874/874 [==============================] - 3s 3ms/step - loss: 5.8500e-05


In [65]:
# Single step evaluation
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions).reshape(predictions.shape[0])

RMSE = np.round(np.sqrt(mean_squared_error(predictions, y_test)), 4)
print('RMSE: '+str(RMSE))
MAE = np.round(mean_absolute_error(predictions, y_test), 4)
print('MAE: '+str(MAE))
MAPE = np.round(mean_absolute_percentage_error(predictions, y_test), 4)
print('MAPE: '+str(MAPE))

35/35 [==============================] - 1s 2ms/step
RMSE: 8.6605
MAE: 7.0904
MAPE: 2.2225


In [67]:
# 5 day ahead model
model = keras.Sequential()
model.add(layers.LSTM(32, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.LSTM(64, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(5))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 5, 32)             4352      
                                                                 
 lstm_19 (LSTM)              (None, 64)                24832     
                                                                 
 dense_18 (Dense)            (None, 25)                1625      
                                                                 
 dense_19 (Dense)            (None, 5)                 130       
                                                                 
Total params: 30,939
Trainable params: 30,939
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 5, epochs=3)

Epoch 1/3
874/874 [==============================] - 5s 3ms/step - loss: 6.2752e-04
Epoch 2/3
874/874 [==============================] - 3s 3ms/step - loss: 5.5344e-05
Epoch 3/3
874/874 [==============================] - 3s 3ms/step - loss: 6.1451e-05


In [69]:
# Multi step evaluation
predictions = model.predict(x_test[:-2])
predictions = scaler.inverse_transform(predictions).reshape(predictions.shape[0]*predictions.shape[1])

y_test_multi = np.array([], dtype=np.float32)
for i in range(len(y_test)):
    y_test_multi = np.append(y_test_multi, y_test[i:i+5])

RMSE = np.round(np.sqrt(mean_squared_error(predictions, y_test_multi)), 4)
print('RMSE: '+str(RMSE))
MAE = np.round(mean_absolute_error(predictions, y_test_multi), 4)
print('MAE: '+str(MAE))
MAPE = np.round(mean_absolute_percentage_error(predictions, y_test_multi), 4)
print('MAPE: '+str(MAPE))

35/35 [==============================] - 1s 2ms/step
RMSE: 10.3296
MAE: 7.8661
MAPE: 2.5045
